### Exercise 5 (Random Variables and Custom Functions)

A random variable maps the sample space to a real line. Let's try sampling from a space with associated mapping to a set of probabilities. In our case, we'll use one vector to store the probabilities and another to store their associated values. Our custom random variable is then defined by the order of the probabilities and values in the vectors (e.g., the first value corresponds to the first probability).

In [ ]:
probs = v(0.3,0.7)

values = v(2.3,6.2)

To draw a value from our custom random variable, we'll first need to draw a random number between 0 and 1. To do so, we'll use the `runif()` function. This is like the `runifInt()` function we used before, but returns a continuous value between some bounds, rather than only integers. The syntax is just like before - the first argument indicates the number of values we want to draw, while the 2nd and 3rd arguments give the bounds for the number we're drawing. We then need to use an index of `1` to extract the single number we're interested in from the list that's returned.

In [ ]:
runif(1,0.0,1.0)[1]

Now we can use `runif()` to draw from our custom random variable. Here's the general logic - we'll stack up the probabilities in our probability vector (recall that they have to sum to 1) and compare the value we drew between 0 and 1 to this stack. For instance, if the probability of our first possible outcome is 0.3 and our random number is 0.15, we would choose this outcome. However, if the random number was 0.35, we would reject the first outcome and continue comparing our number to the cumulative sum of probabilities until we reached a sum greater than 0.35.

![](images/multiDraws.jpg "MultinomialDraws")

This sort of comparison is most naturally done using a `while` loop, which continue executing until their condition is no longer true.

In [ ]:
# Random number between 0 and 1
p = runif(1,0.0,1.0)[1]

# Variables we'll need in our while loop
probSum = 0.0
val = -1.0
i = 1

# Continue looping until probSum is greater than p and we change val
while (val == -1.0){
    probSum += probs[i]
    if (p < probSum){
        val = values[i]
    }
    i += 1
}

# Print out p and the number we drew from our custom random variable
print(p)
print(val)

Now we know how to draw a value from our distribution, but it would be tedious to have to re-write all this code every time we want to do that. Instead, to package together all the steps that are required, we will define a custom function called `drawVal()`. Any user can define their own functions in RevBayes using this syntax:

`function <returnType> <functionName> ( <ArgumentType> <ArgumentName>, ... ) { <CODE> }`.

This might seem a bit daunting at first, but it will be second nature once you've defined a few functions of your own. 

For our purposes, let's use the name `drawVal` for our new function. Because the values we've defined to be drawn are all `Real` numbers, we'll use a return type of `Real`. To make our function flexible, we'll pass it two arguments: our vector of probabilities and our vector of possible values. These each have a type of `Real[]` - a vector of Real numbers. We'll need to give each of these vectors names that we use to access them inside the function, once they've been passed as arguments. For now, let's use `probVec` and `valVec` as their names. At the end of the function, we also need to define a `return` statement. Whenever we run the function, it will pass back to us whatever value(s) is(are) included in the `return` statement.

In [ ]:
# clear()
function Real drawVal (Real[] probVec, Real[] valVec){
 
    p = runif(1,0.0,1.0)[1]

    probSum = 0.0
    val = -1.0
    i = 1

    while (val == -1.0){
        probSum += probVec[i]
        if (p < probSum){
            val = valVec[i]
        }
        i += 1
    }

    return val
    
}

Note that I've put a commented out `clear()` statement above the function definition. This is a bit of hacky workaround, but RevBayes normall prompts the user when they try to overwrite a function name (as would happen if we execute this cell more than once in this notebook). However, because we're using a Jupyter notebook, we can't see or respond to this prompt. Executing `clear()` before defining the function clears all old function definitions (but also all other variables). If you need to redefine the function, you can uncomment this clear() statement. You'll then need to recreate your other variables.

There's also another random __bug__ I discovered while putting this notebook together. If you put a comment (`#`) inside a function definition (or a `for` loop, `if...else` statement, or any other code block) it suppresses further output from RevBayes. This will be fixed in the future, but for now __don't put comments inside code blocks wrapped in {...}__.

In [ ]:
myVal = drawVal(probs,values)
myVal